<a href="https://colab.research.google.com/github/RustamAliyevsHub/Playground/blob/main/GL53%20reconcillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font

# Read the input data from an Excel file
df = pd.read_excel('/content/GL53.xlsx')

# Group the rows by the values in the second column
groups = df.groupby(df.columns[1])

# Define the columns to compare
cols_to_compare = list(range(2, 12)) + list(range(14, df.shape[1]))

# Create a new DataFrame to store the output
output_df = pd.DataFrame(columns=['Group', 'Row Number'] + list(df.columns.difference([df.columns[0], df.columns[11]])))

# Iterate over each group and find the rows with differences
for group_name, group_df in groups:
    # Reset the index to integer values
    group_df = group_df.reset_index(drop=True)
    
    # Use the shift() method to compare the current row with the previous row
    diffs = group_df.iloc[:, cols_to_compare].ne(group_df.iloc[:, cols_to_compare].shift())
    
    # Select only the rows with differences
    diffs = diffs.any(axis=1)
    
    # Check if the group has more than 1 difference
    if sum(diffs) > 1:
        # Add the rows with differences to the output DataFrame
        group_output = group_df.loc[diffs, group_df.columns.difference([df.columns[0], df.columns[11]])].copy()
        group_output.insert(0, 'Group', group_name)
        group_output.insert(1, 'Row Number', group_output.index + 1)
        output_df = pd.concat([output_df, group_output], ignore_index=True)

# Create a new Excel workbook and sheet
workbook = Workbook()
sheet = workbook.active

# Write the DataFrame to the sheet
for row in dataframe_to_rows(output_df, index=False, header=True):
    sheet.append(row)

# Define the font to use for bold cells
bold_font = Font(bold=True)

# Iterate over each row in the sheet, and bold the cells with differences
for row in sheet.iter_rows(min_row=2):
    has_diffs = False
    for cell in row[2:]:
        if cell.value is not None:
            col_index = cell.column - 3  # Adjust for the first two columns that were added
            row_index = row[1].value - 1
            prev_row = group_df.iloc[row_index - 1] if row_index > 0 else None
            if prev_row is not None and prev_row[col_index] == cell.value:
                cell.style = 'Normal'
            else:
                has_diffs = True
                cell.font = bold_font
    if has_diffs:
        row[0].font = bold_font
        row[1].font = bold_font

# Save the workbook to a new Excel file
workbook.save('output.xlsx')
